In [1]:
from __future__ import print_function
import argparse, os, sys, csv, shutil, time, random, operator, pickle, ast, math
import numpy as np
import pandas as pd
from torch.optim import Optimizer
import torch.nn.functional as F
import torch
import pickle
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torch.multiprocessing as mp
import math
sys.path.insert(0,'./../utils/')
from logger import *
from eval import *
from misc import *

from cifar10_normal_train import *
from cifar10_util import *
from adam import Adam
from sgd import SGD

In [2]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
data_loc='/mnt/nfs/work1/amir/vshejwalkar/cifar10_data/'
# load the train dataset

train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

cifar10_train = datasets.CIFAR10(root=data_loc, train=True, download=True, transform=train_transform)

cifar10_test = datasets.CIFAR10(root=data_loc, train=False, download=True, transform=train_transform)

total_tr_len = len(cifar10_train)

X={}
Y=[]
for i in range(len(cifar10_train)):
    data = cifar10_train[i][0].numpy()
    label = cifar10_train[i][1]

    if label in X:
        X[label].append(data)
    else:
        X[label] = []
        X[label].append(data)
        Y.append(label)

for i in range(len(cifar10_test)):
    data = cifar10_test[i][0].numpy()
    label = cifar10_test[i][1]

    if label in X:
        X[label].append(data)
    else:
        X[label] = []
        X[label].append(data)
        Y.append(label)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
n_users=50

print(type(X[0]))

<class 'list'>


In [7]:
import random
def get_label_with_most_sample(labels:list, taken_index:list, initial_num_data_per_label:dict, data:dict):
    max_fraction = -9.99
    result_label = None

    for label in labels:
        
        if label in taken_index: 
            continue
        
        full = initial_num_data_per_label[label]
        current = len(data[label])

        if full == current:
            return label
        else:
            unused_fraction = current / full 
            if unused_fraction > max_fraction:
                max_fraction = unused_fraction
                result_label = label

    return result_label


def split_data_dirichelt(num_user:int, num_class:int, num_data_per_user:int, alpha:int, data:dict):
    
    initial_num_data_per_label = {}
    labels = []

    for label in data:
        initial_num_data_per_label[label] = len(data[label])
        labels.append(label)

    print(initial_num_data_per_label)

    for i in range(num_user):
        alpha_list = [alpha for _ in range(num_class)]
        probs = np.random.dirichlet(alpha_list)

        user_train_data_tensor=[]
        user_train_label_tensor=[]

        taken_index = []

        for prob in probs:
            n_sample = math.floor(prob * num_data_per_user[i])

            # print("n_sample: ", n_sample)
            result_label = get_label_with_most_sample(labels, taken_index, initial_num_data_per_label, data)

            taken_index.append(result_label)
            # print(f"before label {result_label} in data has: ", len(data[result_label]))

            if n_sample > len(data[result_label]):
                sys.exit(f"not enough data for user {num_user}, running out of data with label {label}")

            user_train_data_tensor.extend(data[result_label][:n_sample])
            user_train_label_tensor.extend([result_label] * n_sample)
            
            data[result_label]= data[result_label][n_sample:]


            # print(f"after label {result_label} in data has: ", len(data[result_label]))

        print(f"user {i} has {len(user_train_data_tensor)} train data")
    
    for label in data:
        print(label, len(data[label]))
        

user_train_len= total_tr_len // n_users
split_data_dirichelt(n_users, len(Y), [user_train_len - 100] * n_users, 1, X.copy())


{6: 6000, 9: 6000, 4: 6000, 1: 6000, 2: 6000, 7: 6000, 8: 6000, 3: 6000, 5: 6000, 0: 6000}
user 0 has 895 train data
user 1 has 894 train data
user 2 has 896 train data
user 3 has 895 train data
user 4 has 894 train data
user 5 has 895 train data
user 6 has 894 train data
user 7 has 896 train data
user 8 has 894 train data
user 9 has 895 train data
user 10 has 896 train data
user 11 has 895 train data
user 12 has 894 train data
user 13 has 894 train data
user 14 has 895 train data
user 15 has 895 train data
user 16 has 896 train data
user 17 has 894 train data
user 18 has 894 train data
user 19 has 895 train data
user 20 has 895 train data
user 21 has 895 train data
user 22 has 895 train data
user 23 has 893 train data
user 24 has 894 train data
user 25 has 894 train data
user 26 has 894 train data
user 27 has 897 train data
user 28 has 895 train data
user 29 has 896 train data
user 30 has 895 train data
user 31 has 895 train data
user 32 has 896 train data
user 33 has 894 train data
u